In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.autonotebook import tqdm
from postdocsalary.paths import DATA_DIR, ensure_dir
import urllib.request
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<ipython-input-2-5c81c71768df>:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
university_rankings = pd.read_csv(DATA_DIR / 'university_rankings' / 'cwurData.csv')

In [4]:
university_rankings = university_rankings[university_rankings.country == 'USA']

In [5]:
university_rankings = university_rankings[university_rankings.year == 2015]

In [6]:
len(university_rankings)

229

In [7]:
university_rankings[:3]

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1200,1,Harvard University,USA,1,1,1,1,1,1,1,1.0,3,100.00,2015
1201,2,Stanford University,USA,2,9,2,4,5,3,3,4.0,10,98.66,2015
1202,3,Massachusetts Institute of Technology,USA,3,3,11,2,15,2,2,2.0,1,97.54,2015


In [8]:
len(university_rankings)

229

In [9]:
university_locs = pd.read_csv(DATA_DIR / "university_locations" / 'university_locs.csv')

In [10]:
university_locs[:3]

,UNITID,INSTNM,IALIAS,ADDR,CITY,STABBR,ZIP,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,DUNS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C18BASIC,C18IPUG,C18IPGRD,C18UGPRF,C18ENPRF,C18SZSET,C15BASIC,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,100654,Alabama A & M University,AAMU,4900 Meridian Street,Normal,AL,35762,1,5,"Dr. Andrew Hugine, Jr.",President,2563725000,636001109,197216455,100200,1,www.aamu.edu/,https://www.aamu.edu/admissions-aid/index.html,https://www.aamu.edu/admissions-aid/financial-...,https://www.aamu.edu/admissions-aid/undergradu...,www.aamu.edu/admissions-aid/tuition-fees/net-p...,,,https://www.aamu.edu/administrativeoffices/VAD...,1,1,1,9,1,1,12,1,1,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,18,16,17,10,4,14,18,18,16,1,3,2,-2,-2,26620,1,290,-2,1089,Madison County,105,-86.568502,34.783368,109,1
1,100663,University of Alabama at Birmingham,,Administration Bldg Suite 1070,Birmingham,AL,35294-0110,1,5,Ray L. Watts,President,2059344011,636005396,063690705,105200,1,https://www.uab.edu/,https://www.uab.edu/students/admissions/,https://www.uab.edu/students/paying-for-college,https://ssb.it.uab.edu/pls/sctprod/zsapk003_ug...,https://uab.studentaidcalculator.com/survey.aspx,https://www.uab.edu/students/veterans,https://www.uab.edu/registrar/students,https://www.uab.edu/students/disability/,1,1,1,9,1,1,11,1,2,1,1,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,17,17,9,5,15,15,15,15,2,5,1,The University of Alabama System,101050,13820,1,142,-2,1073,Jefferson County,107,-86.799345,33.505697,95,1
2,100690,Amridge University,Southern Christian University Regions University,1200 Taylor Rd,Montgomery,AL,36117-3553,1,5,Michael C.Turner,President,33438738777550,237034324,126307792,2503400,1,www.amridgeuniversity.edu/,www.amridgeuniversity.edu/admissions/,www.amridgeuniversity.edu/financialaid/,https://www2.amridgeuniversity.edu/Amridge/Log...,www2.amridgeuniversity.edu:9091/,www.amridgeuniversity.edu/admissions/military/,,www.amridgeuniversity.edu/academics/,2,1,2,9,1,1,12,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,20,19,18,5,5,6,20,21,51,2,1,2,-2,-2,33860,1,388,-2,1101,Montgomery County,102,-86.174010,32.362609,126,2


In [11]:
len(university_locs.UNITID.unique())

6440

In [12]:
len(university_locs)

6440

In [13]:
def strip_name(name):
    return ''.join(filter( lambda x: x in '0123456789abcdefghijklmnopqrstuvwxyz', name.lower()))

In [14]:
stripped_institution = np.array([strip_name(i) for i in university_locs.INSTNM.values])

In [15]:
stripped_alias = np.array([strip_name(i) if type(i) == str else np.nan for i in university_locs.IALIAS.values])

In [16]:
import difflib

In [47]:
hand_matched = {
    "universityatbuffalothestateuniversityofnewyork": "universityatbuffalo",
    "sunydownstatemedicalcenter": "SUNYDownstateHealthSciencesUniversity".lower(),
    "lsuhealthsciencescenterneworleans": "LouisianaStateUniversityHealthSciencesCenterNewOrleans".lower(),
    "texasamhealthsciencecenter": "TheUniversityofTexasHealthScienceCenteratHouston".lower(),
    "stateuniversityofnewyorkupstatemedicaluniversity": "UpstateMedicalUniversity".lower(),
    "rutgersthestateuniversityofnewjerseynewark":"RutgersUniversityNewark".lower(),
    "queenscollegecityuniversityofnewyork":"CUNYQueensCollege".lower(),
    "huntercollege": "CUNYHunterCollege".lower(),
    "universityofillinoisaturbanachampaign": strip_name("University of Illinois Urbana-Champaign"),
    "pennsylvaniastateuniversityuniversitypark": strip_name("The Pennsylvania State University"),
    "purdueuniversitywestlafayette":strip_name("Purdue University-Main Campus"),
    "ohiostateuniversitycolumbus":strip_name("Ohio State University-Main Campus"),
    "arizonastateuniversity":strip_name("Arizona State University Campus Immersion"),
    "universityofcoloradodenver":strip_name("University of Colorado Denver/Anschutz Medical Campus"),
    "universityofmassachusettsmedicalschool":strip_name("University of Massachusetts Medical School Worcester"),
    "universityofsouthfloridatampa":strip_name("University of South Florida"),
    "universityofillinoisatchicago":strip_name("University of Illinois Chicago"),
    "northcarolinastateuniversity":strip_name("North Carolina State University at Raleigh"),
    "tulaneuniversity":strip_name("Tulane University of Louisiana"),
    "citycollegeofnewyork":strip_name("CUNY City College"),
    "louisianastateuniversitybatonrouge":strip_name("Louisiana State University and Agricultural & Mechanical College"),
    "collegeofwilliamandmary":strip_name("William & Mary"),
    "washingtonstateuniversitypullman":strip_name("Washington State University"),
    "universityofalabamatuscaloosa":strip_name("The University of Alabama"),
    "oklahomastateuniversitystillwater":strip_name("Oklahoma State University-Main Campus"),
    "georgiaregentsuniversity":strip_name("Augusta University"),
    "miamiuniversity":strip_name("Miami University-Oxford"),
    "texastechuniversityttu":strip_name("Texas Tech University"),
    "universityofnebraskaomaha":strip_name("University of Nebraska at Omaha"),
    "universityofmississippioxfordcampus":strip_name("University of Mississippi"),
    "universityofarkansasfayetteville":strip_name("University of Arkansas"),
    "montanastateuniversitybozeman":strip_name("Montana State University"),
    "kentstateuniversity":strip_name("Kent State University at Kent"),
    "universityofmontanamissoula":strip_name("The University of Montana"),
    "universityofmaineorono":strip_name("University of Maine"),
    "universityofmississippimedicalcenter":strip_name("University of Mississippi"),
    "universityofnorthtexasdenton":strip_name("University of North Texas"),
    "wrightstateuniversitydayton":strip_name("Wright State University-Main Campus"),
    
}

In [52]:
matches = []
for institution in tqdm(university_rankings.institution.values):
    
    lower_institution = ''.join(filter( lambda x: x in '0123456789abcdefghijklmnopqrstuvwxyz', institution.lower()))
    exact_match = True
    
    match = np.where(lower_institution == stripped_institution)[0]
    
    # try the alias
    if len(match) != 1:
        match = np.where(lower_institution == stripped_alias)[0]
        
    # if we need to hand match these universities
    if lower_institution in hand_matched.keys():
        new_name = hand_matched[lower_institution]
        match = np.where(new_name == stripped_institution)[0]
        
    # if no match, print it to make sure we're getting the right university
    if len(match) != 1:
        exact_match = False
        
    # try adding campus
    if len(match) != 1:
        the_lower_institution = lower_institution+'campus'
        match = np.where(the_lower_institution == stripped_institution)[0]
    if len(match) != 1:
        match = np.where(the_lower_institution == stripped_alias)[0]
        
    if len(match) != 1:
        the_lower_institution = lower_institution+'maincampus'
        match = np.where(the_lower_institution == stripped_institution)[0]
    if len(match) != 1:
        match = np.where(the_lower_institution == stripped_alias)[0]
        
    # try adding the
    if len(match) != 1:
        the_lower_institution = 'the'+lower_institution
        match = np.where(the_lower_institution == stripped_institution)[0]
    if len(match) != 1:
        match = np.where(the_lower_institution == stripped_alias)[0]
        
    if exact_match == False:
        print("{}  ({})".format(institution, lower_institution))
    if len(match) == 0:
        breakme
    matches.append(match)

  0%|          | 0/229 [00:00<?, ?it/s]

University of Texas at Austin  (universityoftexasataustin)
University of Washington - Seattle  (universityofwashingtonseattle)
University of Virginia  (universityofvirginia)
Georgia Institute of Technology  (georgiainstituteoftechnology)
University of Texas MD Anderson Cancer Center  (universityoftexasmdandersoncancercenter)
University of Texas at Dallas  (universityoftexasatdallas)
University of Cincinnati  (universityofcincinnati)
University of New Mexico  (universityofnewmexico)
University of Tennessee, Knoxville  (universityoftennesseeknoxville)
University of Texas Health Science Center at San Antonio  (universityoftexashealthsciencecenteratsanantonio)
University of Texas Health Science Center at Houston  (universityoftexashealthsciencecenterathouston)
University of Texas at San Antonio  (universityoftexasatsanantonio)
University of New Hampshire  (universityofnewhampshire)
University of Texas Medical Branch at Galveston  (universityoftexasmedicalbranchatgalveston)
Ohio University 

In [53]:
university_rankings['UNITID'] = university_locs.UNITID.values[matches]

In [54]:
university_rankings = university_rankings.merge(university_locs, how='outer', on='UNITID')

In [55]:
university_rankings["LONGITUDE"] = university_rankings["LONGITUD"]

In [56]:
university_rankings[:5]

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,UNITID,INSTNM,IALIAS,ADDR,CITY,STABBR,ZIP,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,DUNS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C18BASIC,C18IPUG,C18IPGRD,C18UGPRF,C18ENPRF,C18SZSET,C15BASIC,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG,LONGITUDE
0,1.0,Harvard University,USA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,100.00,2015.0,166027,Harvard University,,Massachusetts Hall,Cambridge,MA,02138,25,1,Lawrence S. Bacow,President,6174951000,42103580,082359691|149617367|047006379,215500,1,www.harvard.edu/,college.harvard.edu/admissions,college.harvard.edu/financial-aid,college.harvard.edu/admissions/application-pro...,college.harvard.edu/financial-aid/net-price-ca...,sfs.harvard.edu/veteran-military-benefits,,accessibility.harvard.edu/,2,1,2,9,1,1,11,1,2,2,1,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,8,14,14,6,17,15,15,15,2,5,2,-2,-2,14460,1,148,71650,25017,Middlesex County,2505,-71.118313,42.374471,96,1,-71.118313
1,2.0,Stanford University,USA,2.0,9.0,2.0,4.0,5.0,3.0,3.0,4.0,10.0,98.66,2015.0,243744,Stanford University,,,Stanford,CA,94305,6,8,Marc Tessier-Lavigne,President,6507232300,941156365,009214214,130500,1,www.stanford.edu/,admission.stanford.edu/,financialaid.stanford.edu/,commonapp.org/,www.stanford.edu/dept/finaid/undergrad/how/cal...,exploredegrees.stanford.edu/veterans/,parents.stanford.edu/2015/11/09/stanford-athle...,diversityandaccess.stanford.edu/disability-access,2,1,2,9,1,1,11,1,2,1,1,2,21,1,A,-2,-2,-2,1,1,1,1,1,2,15,11,14,14,6,17,15,15,15,2,4,2,-2,-2,41940,1,488,-2,6085,Santa Clara County,618,-122.167359,37.429434,96,1,-122.167359
2,3.0,Massachusetts Institute of Technology,USA,3.0,3.0,11.0,2.0,15.0,2.0,2.0,2.0,1.0,97.54,2015.0,166683,Massachusetts Institute of Technology,"MIT, M.I.T.",77 Massachusetts Avenue,Cambridge,MA,02139-4307,25,1,L. Rafael Reif,President,6172531000,42103594,001425594,217800,1,web.mit.edu/,https://mitadmissions.org/,https://sfs.mit.edu/,my.mit.edu/,https://sfs.mit.edu/undergraduate-students/our...,https://sfs.mit.edu/undergraduate-students/typ...,,studentlife.mit.edu/das,2,1,2,9,1,1,12,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,15,14,6,17,15,15,15,1,4,2,-2,-2,14460,1,148,71650,25017,Middlesex County,2507,-71.093226,42.359243,96,1,-71.093226
3,6.0,Columbia University,USA,4.0,13.0,6.0,9.0,13.0,13.0,11.0,12.0,4.0,96.14,2015.0,190150,Columbia University in the City of New York,Columbia University,West 116 St and Broadway,New York,NY,10027,36,2,Lee C. Bollinger,President,2128541754,603800000,049179401|621889815,270700,1,www.columbia.edu/,www.columbia.edu/content/admissions,sfs.columbia.edu/,https://www.columbia.edu/content/admissions-of...,cc-seas.financialaid.columbia.edu/estimate-cost,gs.columbia.edu/veterans,,https://health.columbia.edu/content/disability...,2,1,2,9,1,1,11,1,2,2,1,2,11,1,A,-2,-2,-2,1,1,1,1,1,2,15,8,14,15,6,17,15,15,15,2,5,2,-2,-2,35620,1,408,-2,36061,New York County,3610,-73.961885,40.808286,96,2,-73.961885
4,7.0,"University of California, Berkeley",USA,5.0,5.0,21.0,6.0,10.0,4.0,4.0,7.0,29.0,92.25,2015.0,110635,University of California-Berkeley,UC Berkeley,200 California Hall,Berkeley,CA,94720,6,8,Carol Christ,Chancellor,5106426000,946002123,124726725,131200,1,www.berkeley.edu/,admissions.berkeley.edu/,https://financialaid.berkeley.edu/contact-us,admission.universityofcalifornia.edu/apply-now...,https://financialaid.berkeley.edu/sites/defaul...,veteran.berkeley.edu/,ncaa.org/about/resources/research/graduation-r...,dsp.berkeley.edu/,1,1,1,9,1,1,11,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,11,15,15,5,16,15,15

### 

In [58]:
university_rankings.to_pickle(DATA_DIR/ 'university_ranking_statistical_area.pickle')

In [59]:
university_rankings[:3]

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,UNITID,INSTNM,IALIAS,ADDR,CITY,STABBR,ZIP,FIPS,OBEREG,CHFNM,CHFTITLE,GENTELE,EIN,DUNS,OPEID,OPEFLAG,WEBADDR,ADMINURL,FAIDURL,APPLURL,NPRICURL,VETURL,ATHURL,DISAURL,SECTOR,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,HDEGOFR1,DEGGRANT,HBCU,HOSPITAL,MEDICAL,TRIBAL,LOCALE,OPENPUBL,ACT,NEWID,DEATHYR,CLOSEDAT,CYACTIVE,POSTSEC,PSEFLAG,PSET4FLG,RPTMTH,INSTCAT,C18BASIC,C18IPUG,C18IPGRD,C18UGPRF,C18ENPRF,C18SZSET,C15BASIC,CCBASIC,CARNEGIE,LANDGRNT,INSTSIZE,F1SYSTYP,F1SYSNAM,F1SYSCOD,CBSA,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG,LONGITUDE
0,1.0,Harvard University,USA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,100.00,2015.0,166027,Harvard University,,Massachusetts Hall,Cambridge,MA,02138,25,1,Lawrence S. Bacow,President,6174951000,42103580,082359691|149617367|047006379,215500,1,www.harvard.edu/,college.harvard.edu/admissions,college.harvard.edu/financial-aid,college.harvard.edu/admissions/application-pro...,college.harvard.edu/financial-aid/net-price-ca...,sfs.harvard.edu/veteran-military-benefits,,accessibility.harvard.edu/,2,1,2,9,1,1,11,1,2,2,1,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,8,14,14,6,17,15,15,15,2,5,2,-2,-2,14460,1,148,71650,25017,Middlesex County,2505,-71.118313,42.374471,96,1,-71.118313
1,2.0,Stanford University,USA,2.0,9.0,2.0,4.0,5.0,3.0,3.0,4.0,10.0,98.66,2015.0,243744,Stanford University,,,Stanford,CA,94305,6,8,Marc Tessier-Lavigne,President,6507232300,941156365,009214214,130500,1,www.stanford.edu/,admission.stanford.edu/,financialaid.stanford.edu/,commonapp.org/,www.stanford.edu/dept/finaid/undergrad/how/cal...,exploredegrees.stanford.edu/veterans/,parents.stanford.edu/2015/11/09/stanford-athle...,diversityandaccess.stanford.edu/disability-access,2,1,2,9,1,1,11,1,2,1,1,2,21,1,A,-2,-2,-2,1,1,1,1,1,2,15,11,14,14,6,17,15,15,15,2,4,2,-2,-2,41940,1,488,-2,6085,Santa Clara County,618,-122.167359,37.429434,96,1,-122.167359
2,3.0,Massachusetts Institute of Technology,USA,3.0,3.0,11.0,2.0,15.0,2.0,2.0,2.0,1.0,97.54,2015.0,166683,Massachusetts Institute of Technology,"MIT, M.I.T.",77 Massachusetts Avenue,Cambridge,MA,02139-4307,25,1,L. Rafael Reif,President,6172531000,42103594,001425594,217800,1,web.mit.edu/,https://mitadmissions.org/,https://sfs.mit.edu/,my.mit.edu/,https://sfs.mit.edu/undergraduate-students/our...,https://sfs.mit.edu/undergraduate-students/typ...,,studentlife.mit.edu/das,2,1,2,9,1,1,12,1,2,2,2,2,12,1,A,-2,-2,-2,1,1,1,1,1,2,15,14,15,14,6,17,15,15,15,1,4,2,-2,-2,14460,1,148,71650,25017,Middlesex County,2507,-71.093226,42.359243,96,1,-71.093226


### Get names of statistical area

In [78]:
stat_area_df = pd.read_excel(DATA_DIR / "statistical_areas" / "statistical_areas2.xlsx", skiprows=2)[:-4]
stat_area_df[:3]

,CBSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Principal City Name,FIPS State Code,FIPS Place Code
0,10100,"Aberdeen, SD",Micropolitan Statistical Area,Aberdeen,46.0,100.0
1,10140,"Aberdeen, WA",Micropolitan Statistical Area,Aberdeen,53.0,100.0
2,10180,"Abilene, TX",Metropolitan Statistical Area,Abilene,48.0,1000.0


In [79]:
stat_area_df["CBSA Code"].values

array(['10100', '10140', '10180', ..., '49740', '49780', '49820'],
      dtype=object)

In [82]:
stat_area_df['CBSA Code'] = stat_area_df['CBSA Code'].astype(int)

In [102]:
CBSA_NAMES = []
for idx, row in university_rankings.iterrows():
    if row.CBSA in stat_area_df['CBSA Code'].values:
        CBSA_TITLE = stat_area_df[(row.CBSA == stat_area_df['CBSA Code'].values)]['CBSA Title'].values[0]
    else:
        print(row.INSTNM)
        CBSA_TITLE = np.nan
    CBSA_NAMES.append(CBSA_TITLE)

Judson College
University of West Alabama
Lurleen B Wallace Community College
Marion Military Institute
Reid State Technical College
Tuskegee University
Alaska Vocational Technical Center
Dine College
University of Arkansas at Monticello
Black River Technical College
Cossatot Community College of the University of Arkansas
Ozarka College
University of the Ozarks
University of Arkansas Community College-Morrilton
University of Arkansas Community College Rich Mountain
Williams Baptist University
Feather River Community College District
College of the Siskiyous
Adams State University
Colorado Northwestern Community College
Technical College of the Rockies
Lamar Community College
Otero Junior College
Trinidad State Junior College
Western Colorado University
The Baptist College of Florida
North Florida Technical College
Chipola College
North Florida College
Riveroak Technical College
Big Bend Technical College
Florida Panhandle Technical College
Andrew College
East Georgia State College
Emm

In [103]:
university_rankings["CBSA_NAME"] = CBSA_NAMES

In [108]:
university_rankings.to_pickle(DATA_DIR/ 'university_ranking_statistical_area.pickle')

In [109]:
len(university_rankings)

6442